In [ ]:
#4bi) Adagrad GD-m OLS

In [ ]:
# Importing various packages
from random import random, seed
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
import sys
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split

def MSE(y_data,y_model):
    n = np.size(y_model)
    return np.sum((y_data-y_model)**2)/n

#grid size
n = 200 

# Make data set.
x = np.linspace(0,(np.pi)/2,n).reshape(-1, 1)

#Northern H. parameters
s0 = 1
s2 = -0.473
a0 = 0.675
a2 = -0.192
i2 = -0.165

#flux function (eqn. (14) from Stone_1978)
y = 0.5*(s0*a2+s2*a0+(2/7)*s2*a2-i2)*((np.sin(x))**3-np.sin(x))

#noisy flux function
y_noisy = np.random.normal(y, abs(y*0.05)) 

#polynomial fit
degree=6
poly = PolynomialFeatures(degree=degree)
X = poly.fit_transform(x)

# Hessian matrix
H = (2.0/n)* X.T @ X

# Get the eigenvalues
EigValues, EigVectors = np.linalg.eig(H)

beta_OLS = np.random.randn(degree+1,1)

Niterations = 1000000 
  
# we vary the value of learning rate
ngammas = 10 

gammas = np.linspace(1.0/np.max(EigValues), 0.03, ngammas)

MSE_OLS_GD_m_ada = np.zeros(ngammas)   

for g in range(ngammas):
    gamma = gammas[g]

    # Including AdaGrad parameter to avoid possible division by zero
    delta  = 1e-8

    change_OLS = 0.0
    delta_momentum = 0.3

    # The outer product is calculated from scratch for each epoch
    Giter = np.zeros(shape=(degree+1,degree+1))

    for iter in range(Niterations):
        gradient_OLS = (2.0/n)*X.T @ (X @ beta_OLS-y_noisy)

        # Calculate the outer product of the gradients
        Giter +=gradient_OLS @ gradient_OLS.T

        # Simpler algorithm with only diagonal elements
        Ginverse = np.c_[gamma/(delta+np.sqrt(np.diagonal(Giter)))]

        # compute update
        update = np.multiply(Ginverse,gradient_OLS)+delta_momentum*change_OLS

        beta_OLS -= update

        # save the change
        change_OLS = update

    y_OLS_GD_m_ada= X@beta_OLS
    
    MSE_OLS_GD_m_ada[g] = MSE(y_noisy, y_OLS_GD_m_ada)
    
print('method MSE=', MSE_OLS_GD_m_ada)  
print('β=', beta_OLS)
    
plt.figure()
plt.plot(gammas, MSE_OLS_GD_m_ada, label='MSE OLS GD-m Adagrad')
plt.xlabel('γ')
plt.ylabel('MSE')
plt.legend()
plt.show()    

plt.figure()
plt.plot(x*180/np.pi, y_noisy, 'ro', label='data')
plt.plot(x*180/np.pi, y_OLS_GD_m_ada, label='Adagrad GD-m OLS')
plt.xlabel('latitude [degrees]')
plt.ylabel('f')
plt.legend()
plt.show()

#Calculating the time processing time
import time
start_time = time.time()
print("execution time=", (time.time() - start_time), 's')

In [ ]:
#4bii) Adagrad GD-m Ridge

In [ ]:
#Ridge parameter
#Ridge parameter
nlambdas = 12 
lambdas = np.logspace(-5, 0, nlambdas) 

#leaning rate constants
Niterations = 100000  
ngammas = 10 
gammas = np.linspace(1.0/np.max(EigValues), 0.03, ngammas) 

y_Ridge_GD_m_ada_array= np.zeros((nlambdas, ngammas), dtype=object)
MSE_Ridge_GD_m_ada = np.zeros((nlambdas, ngammas))

for l in range(nlambdas):
    lmbda = lambdas[l]
    beta_Ridge = np.random.randn(degree+1,1)    

    # We vary the learning rate
    for g in range(ngammas):
        gamma = gammas[g]

        # Including AdaGrad parameter to avoid possible division by zero
        delta  = 1e-8

        Giter = np.zeros(shape=(degree+1,degree+1))

        change_Ridge = 0.0
        delta_momentum = 0.3

        for iter in range(Niterations):
            gradient_Ridge = (2.0/n)*X.T @ (X @ (beta_Ridge)-y_noisy)+2*lmbda*beta_Ridge

            # Calculate the outer product of the gradients
            Giter +=gradient_Ridge @ gradient_Ridge.T

            # Simpler algorithm with only diagonal elements
            Ginverse = np.c_[gamma/(delta+np.sqrt(np.diagonal(Giter)))]

            # compute update
            update = np.multiply(Ginverse,gradient_Ridge)+delta_momentum*change_Ridge
            beta_Ridge -= update

            # save the change
            change_Ridge = update

        y_Ridge_GD_m_ada= X@beta_Ridge
        
        y_Ridge_GD_m_ada_array[l, g] = y_Ridge_GD_m_ada

        MSE_Ridge_GD_m_ada[l, g]= MSE(y_noisy, y_Ridge_GD_m_ada)        

#finding the minimum value of the MSE
MSE_Ridge_GD_m_ada_optimal = np.min(MSE_Ridge_GD_m_ada)
conditon = (MSE_Ridge_GD_m_ada == MSE_Ridge_GD_m_ada_optimal)
#the l, g for which we have the minimum MSE
result = np.where(conditon)
print(result)

print('method MSE=', MSE_Ridge_GD_m_ada_optimal)  

#saving the y that gives the optimal MSE
y_Ridge_GD_m_ada_optimal= y_Ridge_GD_m_ada_array[result]
y_Ridge_GD_m_ada_optimal=(y_Ridge_GD_m_ada_optimal[0])
#print(y_Ridge_GD_optimal)
        
import pandas as pd
import seaborn as sns

pd.DataFrame(MSE_Ridge_GD_m_ada)
mse_data_ridge = pd.DataFrame(MSE_Ridge_GD_m_ada)

plt.figure(figsize=(15,10))
sns.heatmap(data=mse_data_ridge, annot=True,  fmt=".1e", cmap="crest")
plt.xlabel("γ")
plt.ylabel("λ")
plt.show()

plt.figure()
plt.plot(x*180/np.pi, y_noisy, 'ro', label='data')
plt.plot(x*180/np.pi, y_OLS_GD_m_ada, label='Adagrad GD-m OLS')
plt.plot(x*180/np.pi, y_Ridge_GD_m_ada_optimal, 'c-', label='Adagrad GD-m Ridge optimal')
plt.xlabel('latitude [degrees]')
plt.ylabel('f')
plt.legend()
plt.show()

#Calculating the time processing time
import time
start_time = time.time()
print("execution time=", (time.time() - start_time), 's')